1<center><b><h4>Look at the big picture:</h4></b></center>
<hr>


El objetivo del desafio es clasificar los fármacos en función de su actividad biológica para identificar ciertas proteínas que están asociadas con una enfermedad específica y luego desarrollar moléculas que puedan apuntar a esas proteínas.

El conjunto de datos describe las respuestas de 100 tipos diferentes de células humanas a varios fármacos. Estos patrones de respuesta se utilizarán para clasificar la respuesta del MoA.

Es un problema de clasificación de etiquetas múltiples.
Los medicamentos pueden tener múltiples anotaciones de MoA que describen respuestas binarias de diferentes tipos de células de diferentes maneras. La métrica de evaluación es la pérdida logarítmica media por columnas.

Los datos vienen en forma familiar de archivos de prueba y de entrenamiento. A diferencia de otras competiciones, aquí tenemos dos archivos separados para los predictores de entrenamiento (train_features.csv) y los objetivos (train_targets_scored.csv). Cada fila corresponde a un tratamiento específico.

El objetivo específico es predecir las probabilidades de la clase train_targets_scored.csv para el archivo de prueba test_features.csv. Tenga en cuenta que la tabla de clasificación pública se basa en aproximadamente el 25% de los datos de esta prueba, y el 75% se asigna al LB privado. ¡Empecemos!


 &nbsp;

<center><h3> Get the Data <h3></center>
2.1<hr><br>  
en esta estapa se prepara la zona de trabajo ademas de importar librerias y los datos para empezar a analizarlos.

En este caso, se importo las librerias (pandas, numpy, seaborn, matplotlib, sklearn)
<br>
<br>
<h6></h6>
<hr>

In [ ]:
from sklearn.datasets import *
import pandas as pd
import matplotlib.pyplot as  plt
import seaborn as sns
import numpy as np
import os

2.2 <center>Create a Test Set</center>
<hr><br>

Ya una vez importadas las herramientas ahora si podemos traer los datos para comenzar a analizarlos.

importa los CSV, el reto ya separa los datos de entrenamiento con los de testeo asi que nos podemos saltar la parte de dividir el dataset. 

<br><hr>

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
       
        if(filename=="train_targets_scored.csv"):
            train_target = pd.read_csv(os.path.join(dirname, filename))
            print("imported %s"%(os.path.join(dirname, filename)))
        if(filename=="train_features.csv"):
            train = pd.read_csv(os.path.join(dirname, filename))
            print("imported %s"%(os.path.join(dirname, filename)))
        if(filename=="test_features.csv"):
            test = pd.read_csv(os.path.join(dirname, filename))
            print("imported %s"%(os.path.join(dirname, filename)))


<center><h3> Discover and visualize the data to gain insights
<h3></center>

3
<hr>

En este punto se pueden ver todas las caracteristicas y conocer un poco como se distribuyen los datos para esto podemos representarlas por medio de histograma y el describe para entender los datos.Tambien hay que tratar de no caer en el Data snooping.

<br>

train_features.csv: funciones para el conjunto de entrenamiento. Las características g- significan datos de expresión génica y c- significan datos de viabilidad celular. cp_type indica muestras tratadas con un compuesto (cp_vehicle) o con una perturbación de control (ctrl_vehicle); las perturbaciones de control no tienen MoA; cp_time y cp_dose indican la duración del tratamiento (24, 48, 72 horas) y la dosis (alta o baja).

In [ ]:
train.info()

In [ ]:
train.columns

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
train.describe().T

Los datos no contienen valores nulos dentro de el dataframe de entrenamento.

In [ ]:
train.corr()

MIramos las correlaciones de 20 variables seleccionadas al azar

In [ ]:
train_columns = train.columns.to_list()

g_list = [
    i for i in train_columns if i.startswith('g-')
]

c_list = [
    i for i in train_columns if i.startswith('c-')
]
columns = g_list + c_list
for_correlation = list(set([columns[np.random.randint(0, len(columns)-1)] for i in range(200)]))[:20]
data = train[for_correlation]

f = plt.figure(
    figsize=(18, 18)
)

plt.figure(figsize=(20,10))
sns.heatmap(data.corr(),annot=True)


In [ ]:
train.describe()

<center><h3> Prepare the Data for Machine Learning Algorithms
<h3></center>

4
<hr>

se realiza Data cleaning y escalamiento de los datos y se preparan los datos y para poder ser entrenados.


<br>

en este caso, el dataset biene completo, no cuenta con valones null como muestra el describe() en count y en la grafica de que nos entrega missingno 
<hr>

In [ ]:
train.describe()

In [ ]:
import missingno as msno
msno.matrix(train)

In [ ]:
data.hist(bins=50, figsize=(20,15))

<center><h3> Select and Train a Model <h3></center>
5<hr><br>  

En esta etapa se preparan los modelos, se realiza GridSearch o RandomGridSearch y ademas de ahacer CV y otros metodos de validacion para detectar que el modelo no tenga el caso de overfiting. Tambien, una vez todo este correcto se hacen las predicciones con los datos de testeo y se evalua el score.
<br>
<br>
<h6></h6>

Para este punto, selecionamos un modelo de clasificacion que se adapte a las necesidades del problema, un metodo de clasificación llamada XGBClassifier esta y implementando tambien RandomGridSearch.
<hr>

Eliminar variables de tipo objeto

In [ ]:
x_train=train.drop(['sig_id'], axis=1)
x_train=x_train.drop(['cp_type'], axis=1)
x_train=x_train.drop(['cp_dose'], axis=1)
y_train=train.drop(['sig_id'], axis=1)
y_test=test.drop(['sig_id'], axis=1)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing
from category_encoders import CountEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import *

In [ ]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))
pipe = Pipeline([('encode', CountEncoder(cols=[0, 2])),
                ('classify', classifier)
               ])
x_train_trans = pipe.fit_transform(x_train)
x_train_trans_df = pd.DataFrame(x_train_trans, columns=x_train.columns,
                          index=x_train.index)

In [ ]:
x_train_trans_df = pd.DataFrame(x_train_trans, columns=x_train_numeric.columns,
                          index=x_train_numeric.index)

In [ ]:
x_train_trans_df.fit(x_train, y_train)